In [1]:
import torch
from sentence_transformers import SentenceTransformer

import nltk
import pandas as pd
import numpy as np
import sqlite3
import dill as pickle

In [2]:
conn = sqlite3.connect("../cord.db")

In [3]:
df = pd.read_sql_query("SELECT title, abstract, authors, body_text FROM cord19", conn)

In [4]:
df.head()

,title,abstract,authors,body_text
0,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...,"Madani, Tariq A; Al-Ghamdi, Aisha A",
1,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...,"Vliet, Albert van der; Eiserich, Jason P; Cros...",
2,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...,"Crouch, Erika C",
3,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",
4,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",


In [5]:
titles = df['title']
authors = df['authors']
document = df['abstract'] + df['body_text']

In [6]:
model = SentenceTransformer('monologg/biobert_v1.1_pubmed')

Downloading: 100%|██████████| 433/433 [00:00<00:00, 237kB/s]
Downloading: 100%|██████████| 436M/436M [03:43<00:00, 1.95MB/s]    
Downloading: 100%|██████████| 112/112 [00:00<00:00, 41.2kB/s]
Downloading: 100%|██████████| 51.0/51.0 [00:00<00:00, 19.4kB/s]
Downloading: 100%|██████████| 213k/213k [00:01<00:00, 164kB/s]  
Some weights of the model checkpoint at /home/harry/.cache/torch/sentence_transformers/monologg_biobert_v1.1_pubmed were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- 

In [7]:
# with open('../data/biobert.pickle', 'rb') as file :
#     model = pickle.load(file)

if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))

In [8]:
title_embeddings = model.encode(titles, show_progress_bar=True)

Batches: 100%|██████████| 144/144 [03:46<00:00,  1.57s/it]


In [9]:
document_embeddings = model.encode(document, show_progress_bar=True) # 5 mins

Batches: 100%|██████████| 144/144 [45:28<00:00, 18.95s/it]


In [11]:
with open('../data/title_embeddings.pickle', 'wb') as file :
    pickle.dump(title_embeddings, file)

In [ ]:
with open('../data/document_embeddings.pickle', 'wb') as file :
    pickle.dump(document_embeddings, file)

In [ ]:
#for relevant_factors table
df = pd.read_sql_query("SELECT Study, Factors, Excerpt FROM relevant_factors", conn)

study_title = df['Study'].map(str)
study_text = df['Factors'].map(str) + df['Excerpt'].map(str)

study_title_embeddings = model.encode(study_title, show_progress_bar=True) 
study_text_embeddings = model.encode(study_text, show_progress_bar=True)

with open('../data/study_title_embeddings.pickle', 'wb') as file :
    pickle.dump(study_title_embeddings, file)

with open('../data/study_text_embeddings.pickle', 'wb') as file :
    pickle.dump(study_text_embeddings, file)


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
#for symptoms table
df = pd.read_sql_query("SELECT Study, Excerpt FROM symptoms", conn)

symptoms_study_title = df['Study'].map(str)
symptoms_study_text = df['Excerpt'].map(str)

symptoms_study_title_embeddings = model.encode(symptoms_study_title, show_progress_bar=True) 
symptoms_study_text_embeddings = model.encode(symptoms_study_text, show_progress_bar=True)

with open('../data/symptoms_study_title_embeddings.pickle', 'wb') as file :
    pickle.dump(symptoms_study_title_embeddings, file)

with open('../data/symptoms_study_text_embeddings.pickle', 'wb') as file :
    pickle.dump(symptoms_study_text_embeddings, file)


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]